In [1]:
# Imports
import pandas as pd
import os
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import re
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

## Read from jmeter txt File ##

In [2]:
# If running from Binder uncomment following line
# fname = "./Trust_testResults.txt"

# If using google colab or azure notebook rename the path in variable : fname
fname = "C:/Jmeter_Dev/Results/Trust_testResults.txt"
f = open(fname)
lines = f.readlines()
rng = len(lines)
my_lt = []
for i in range(2, rng, 4):
    if i < rng:
        x = lines[i]
        my_lt.append(re.split(r'\t+', x))
    
f.close()

## Store DataFrames in List ##

In [3]:
def hms_to_sec(val):
    l = val.split(':')
    sec = int(l[1])*60 + int(l[2])
    return sec

result = []

for i in my_lt:
    trust_with_task_list = i[0].split(',')
    task_trust_dict = {val.strip().split('-')[1].strip():val.strip().split('-')[0].strip()  for val in trust_with_task_list}

    task_with_timestamp = i[1].split(',')
    task_timestamp_dict = {val.strip().split('-')[1].strip():val.strip().split('-')[0].strip()  for val in task_with_timestamp}

    df1 = pd.DataFrame(task_trust_dict.values(), index=task_trust_dict.keys())
    df1.rename(columns={0:"TrustID"}, inplace=True)

    df2 = pd.DataFrame(task_timestamp_dict.values(), index=task_timestamp_dict.keys())
    df2.rename(columns={0:"Time Taken (h:m:s)"}, inplace=True)

    final_df = df1.join(df2)

    final_df["Time(in sec)"] = final_df["Time Taken (h:m:s)"].apply(hms_to_sec)
    
    result.append(final_df)


## Helper Functions ##

In [4]:
def make_chart(df):
    selection = alt.selection_interval(encodings=['x'])

    colorCondition = alt.condition(selection, alt.value('lightgreen'), alt.value('gray'))

    x = alt.Chart(df).mark_bar().encode(
        x=alt.X('TrustID', sort='-y'),
        y='Time(in sec)',
        tooltip = ['TrustID', 'Time Taken (h:m:s)'],
        color=colorCondition
    ).interactive().add_selection(selection).properties(
        width=500,
        height=400
    )
    
    return x

def trust_frame(result_frames, trust_id):
    #trust_id = "ODART20181"
    trust_dframe = None
    for dframe in result_frames:
        trust_row = dframe[dframe['TrustID'] == trust_id]
        if trust_dframe is None:
            trust_dframe = trust_row
        else:
            trust_dframe = pd.concat([trust_dframe, trust_row], axis=0)
        
    return trust_dframe

def trust_chart(trust_frame, trust_id):
    x = alt.Chart(trust_frame.reset_index()).mark_bar().encode(
        x='index',
        y='Time(in sec)',
        color=alt.value('lightgreen'),
    ).properties(title=trust_id, width=300)
    
    return x

## Widget for Iterations ##

In [5]:
iteration_list = ['Iteration-' + str(i) for i in range(1, len(result)+1)]

w = widgets.Dropdown(
    options=iteration_list,
    value=iteration_list[0],
    description='Iteration:',
)

display(w)

def display_fnc(iter_list = iteration_list):
    for i in iter_list:
        get_index = str(i).split('-')[1]
        my_index=int(get_index)-1
        make_chart(result[my_index]).display()
        display(result[my_index].sort_values('Time(in sec)', ascending=False))

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        get_index = str(change['new']).split('-')[1]
        clear_output()
        display(w)
        display(make_chart(result[int(get_index)-1]))
        display(result[int(get_index)-1].sort_values('Time(in sec)', ascending=False))

w.observe(on_change)

Dropdown(description='Iteration:', index=1, options=('Iteration-1', 'Iteration-2', 'Iteration-3', 'Iteration-4…

alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130266,OMFIT2019A,0:4:6,246
130263,OMFIT20182,0:3:57,237
130262,OMFIT20163,0:3:56,236
130264,OMFIT20191,0:3:56,236
130265,OMFIT20192,0:2:57,177
130269,OMFIT20211,0:2:57,177
130260,OMFIT20181,0:2:57,177
130267,OMFIT20201,0:2:57,177
130271,SFT2017A,0:2:50,170
130258,OMFIT20153,0:2:50,170


## Widget for Individual Trusts ##

In [6]:
trust_iteration_list = result[0].TrustID.tolist()

z = widgets.Dropdown(
    options=trust_iteration_list,
    value=trust_iteration_list[0],
    description='Trust:',
)

display(z)

def display_trust_fnc(iter_list = trust_iteration_list):
    for i in iter_list:
        get_trust = i
        trust_df = trust_frame(result, get_trust)
        display(trust_chart(trust_df, get_trust))
        display(trust_df)

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        get_trust = change['new']
        clear_output()
        display(z)
        trust_df = trust_frame(result, get_trust)
        display(trust_chart(trust_df, get_trust))
        display(trust_df)

z.observe(on_change)

Dropdown(description='Trust:', index=2, options=('ODART20181', 'ODART20191', 'OMFIT20182', 'OMFIT20192', 'OMFI…

alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130207,OMFIT20182,0:3:45,225
130263,OMFIT20182,0:3:57,237
130328,OMFIT20182,0:2:45,165
130384,OMFIT20182,0:2:36,156


# In case above cells dropdowns dont work #

### Run the following cells ###

### All Iterations

In [7]:
display_fnc()

alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130206,OMFIT20163,0:4:39,279
130202,OMFIT20153,0:4:36,276
130210,OMFIT2019A,0:4:31,271
130214,SFT2015B,0:4:31,271
130203,ODART20181,0:3:46,226
130205,ODART20191,0:3:46,226
130207,OMFIT20182,0:3:45,225
130204,OMFIT20181,0:3:45,225
130208,OMFIT20191,0:3:45,225
130213,OMFIT20211,0:3:43,223


alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130266,OMFIT2019A,0:4:6,246
130263,OMFIT20182,0:3:57,237
130262,OMFIT20163,0:3:56,236
130264,OMFIT20191,0:3:56,236
130265,OMFIT20192,0:2:57,177
130269,OMFIT20211,0:2:57,177
130260,OMFIT20181,0:2:57,177
130267,OMFIT20201,0:2:57,177
130271,SFT2017A,0:2:50,170
130258,OMFIT20153,0:2:50,170


alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130334,OMFIT20211,0:4:8,248
130331,OMFIT2019A,0:4:8,248
130332,OMFIT20201,0:4:8,248
130333,OMFIT20202,0:4:7,247
130323,OMFIT20153,0:2:45,165
130327,OMFIT20163,0:2:45,165
130324,ODART20181,0:2:45,165
130328,OMFIT20182,0:2:45,165
130335,SFT2015B,0:2:45,165
130329,OMFIT20191,0:2:44,164


alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130383,OMFIT20163,0:3:55,235
130387,OMFIT2019A,0:3:45,225
130391,SFT2015B,0:3:45,225
130392,SFT2017A,0:3:45,225
130379,OMFIT20153,0:2:42,162
130380,ODART20181,0:2:42,162
130381,OMFIT20181,0:2:42,162
130382,ODART20191,0:2:42,162
130388,OMFIT20201,0:2:36,156
130384,OMFIT20182,0:2:36,156


#### All Trusts

In [8]:
display_trust_fnc()

alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130203,ODART20181,0:3:46,226
130259,ODART20181,0:2:50,170
130324,ODART20181,0:2:45,165
130380,ODART20181,0:2:42,162


alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130205,ODART20191,0:3:46,226
130261,ODART20191,0:2:45,165
130326,ODART20191,0:2:44,164
130382,ODART20191,0:2:42,162


alt.Chart(...)

...

,TrustID,Time Taken (h:m:s),Time(in sec)
130208,OMFIT20191,0:3:45,225
130264,OMFIT20191,0:3:56,236
130329,OMFIT20191,0:2:44,164
130385,OMFIT20191,0:2:36,156


alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130213,OMFIT20211,0:3:43,223
130269,OMFIT20211,0:2:57,177
130334,OMFIT20211,0:4:8,248
130390,OMFIT20211,0:2:35,155


alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130214,SFT2015B,0:4:31,271
130270,SFT2015B,0:2:50,170
130335,SFT2015B,0:2:45,165
130391,SFT2015B,0:3:45,225
